In [2]:
import os
import tensorflow as tf

# Clear old graphs (avoids OOM)
tf.keras.backend.clear_session()

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

print("TensorFlow:", tf.__version__)
print("GPUs:", gpus)


TensorFlow: 2.10.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
BASE_DIR = r"D:\plant_disease_prediction\dataset\grape"

TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR   = os.path.join(BASE_DIR, "test")  # test used as validation

IMAGE_SIZE = (224,224)
BATCH_SIZE = 32
EPOCHS = 30


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

NUM_CLASSES = train_generator.num_classes
print("Classes:", train_generator.class_indices)


Found 7222 images belonging to 4 classes.
Found 1805 images belonging to 4 classes.
Classes: {'Black Rot': 0, 'ESCA': 1, 'Healthy': 2, 'Leaf Blight': 3}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense,
    Flatten, Dropout, BatchNormalization
)

model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(256, (3,3), activation="relu"),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),

    Dense(NUM_CLASSES, activation="softmax")
])


In [6]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 109, 109, 64)     256       
 hNormalization)                                                 
                                                        

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "best_grape_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


In [8]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)


Epoch 1/30
226/226 [==============================] - ETA: 0s - loss: 0.5263 - accuracy: 0.8283
Epoch 1: val_accuracy improved from -inf to 0.36620, saving model to best_grape_model.h5
226/226 [==============================] - 170s 689ms/step - loss: 0.5263 - accuracy: 0.8283 - val_loss: 15.5916 - val_accuracy: 0.3662
Epoch 2/30
226/226 [==============================] - ETA: 0s - loss: 0.2467 - accuracy: 0.9046
Epoch 2: val_accuracy improved from 0.36620 to 0.46482, saving model to best_grape_model.h5
226/226 [==============================] - 54s 238ms/step - loss: 0.2467 - accuracy: 0.9046 - val_loss: 7.3893 - val_accuracy: 0.4648
Epoch 3/30
226/226 [==============================] - ETA: 0s - loss: 0.1827 - accuracy: 0.9310
Epoch 3: val_accuracy improved from 0.46482 to 0.74349, saving model to best_grape_model.h5
226/226 [==============================] - 53s 235ms/step - loss: 0.1827 - accuracy: 0.9310 - val_loss: 1.5591 - val_accuracy: 0.7435
Epoch 4/30
226/226 [===============